In [11]:
import geopy
import requests
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
import logging
import pytz
import threading
import time
import sqlite3

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Conectar ao arquivo .db (ele será criado se não existir)
conn = sqlite3.connect('dados_onibus.db')
cursor = conn.cursor()

# Criar tabela (se ainda não existir)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS onibus (
        COD INTEGER,
        REFRESH TEXT,
        LAT_IN_TIME REAL,
        LON_IN_TIME REAL,
        CODIGOLINHA INTEGER,
        ADAPT INTEGER,
        TIPO_VEIC INTEGER,
        TABELA TEXT,
        SITUACAO TEXT,
        SITUACAO2 TEXT,
        SENT TEXT,
        TCOUNT INTEGER,
        SENTIDO_IN_TIME TEXT,
        HORA TEXT,
        FLAG_PROCES INTEGER
    )
''')
conn.commit()

# Função para buscar e processar os dados de uma linha de ônibus
def buscar_e_processar_dados(linha):
    url_base = f'https://transporteservico.urbs.curitiba.pr.gov.br/getVeiculos.php?linha={linha:03}&c=821f0'
    try:
        response = requests.get(url_base, timeout=10)
        response.raise_for_status()

        timezone_sp = pytz.timezone('America/Sao_Paulo')
        hora_online = datetime.now(timezone_sp).strftime("%Y-%m-%d %H:%M:%S")
        dados_json = response.json()

        if not dados_json or not isinstance(dados_json, dict):
            return None

        codigos_onibus = [{
            "COD": valor['COD'],
            "REFRESH": valor['REFRESH'],
            "LAT_IN_TIME": valor['LAT'],
            "LON_IN_TIME": valor['LON'],
            "CODIGOLINHA": int(valor['CODIGOLINHA']),
            "ADAPT": int(valor['ADAPT']),
            "TIPO_VEIC": int(valor['TIPO_VEIC']),
            "TABELA": valor['TABELA'],
            "SITUACAO": valor['SITUACAO'],
            "SITUACAO2": valor['SITUACAO2'],
            "SENT": valor['SENT'],
            "TCOUNT": valor.get('TCOUNT', 0),
            "SENTIDO_IN_TIME": valor['SENTIDO']
        } for chave, valor in dados_json.items()]

        df_codigos_onibus = pd.DataFrame(codigos_onibus)
        if not df_codigos_onibus.empty:
            df_codigos_onibus['HORA'] = hora_online
        else:
            logging.warning("Nenhum ônibus encontrado para a linha %s", linha)
            return None

        return df_codigos_onibus

    except requests.Timeout:
        logging.error("Timeout: A conexão para a linha %s demorou mais de 10 segundos", linha)
        return None
    except requests.RequestException as e:
        logging.error("Erro na requisição para a linha %s: %s", linha, e)
        return None
    except Exception as e:
        logging.error("Erro ao processar a linha %s: %s", linha, e)
        return None

# Função para processar cada linha e armazenar os resultados em uma lista
def processar_linha(linha, df_linhas, result_list):
    df_result = buscar_e_processar_dados(linha)
    if df_result is not None:
        result_list.append(df_result)

# Função para gravar os dados no banco de dados SQLite
def gravar_no_banco(df_result):
    for index, row in df_result.iterrows():
        cursor.execute('''
            INSERT INTO onibus (COD, REFRESH, LAT_IN_TIME, LON_IN_TIME, CODIGOLINHA, ADAPT, TIPO_VEIC, TABELA, 
                                SITUACAO, SITUACAO2, SENT, TCOUNT, SENTIDO_IN_TIME, HORA, FLAG_PROCES) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['COD'], row['REFRESH'], row['LAT_IN_TIME'], row['LON_IN_TIME'], row['CODIGOLINHA'],
            row['ADAPT'], row['TIPO_VEIC'], row['TABELA'], row['SITUACAO'], row['SITUACAO2'], 
            row['SENT'], row['TCOUNT'], row['SENTIDO_IN_TIME'], row['HORA'], 0
        ))
    conn.commit()
        # Remover as linhas que foram inseridas no banco de dados do DataFrame global
    global df_concatenado
    df_concatenado = df_concatenado.drop(df_result.index).reset_index(drop=True)

# Carregar os dados dos pontos da linha
df_linhas = pd.read_csv('df_pontos.csv')

# Obter os códigos únicos das linhas
linhas_unicas = df_linhas['COD'].unique()

# DataFrame final para armazenar os resultados concatenados
df_concatenado = pd.DataFrame()

# Função para gerenciar threads e processamento de dados
def processar_linhas_com_threads(linhas_unicas, df_linhas):
    global df_concatenado
    i = 0
    while True:
        result_list = []

        # Criar e iniciar threads para cada linha
        threads = [
            threading.Thread(target=processar_linha, args=(linha, df_linhas, result_list)) 
            for linha in linhas_unicas
        ]

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

        # Concatenar resultados e gravar no banco de dados
        if result_list:
            df_concatenado = pd.concat([df_concatenado] + result_list, ignore_index=True)
            gravar_no_banco(df_concatenado)

        logging.info('Número atual da iteração: %s', i)
        break
        i += 1

        # Espera 60 segundos antes da próxima iteração
        time.sleep(60)

# Inicializar o processamento com threads
try:
    processar_linhas_com_threads(linhas_unicas, df_linhas)
finally:
    # Fechar a conexão com o banco de dados ao final
    conn.close()


2024-10-16 22:16:12,323 - ERROR - Timeout: A conexão para a linha 231 demorou mais de 10 segundos
2024-10-16 22:16:16,915 - ERROR - Timeout: A conexão para a linha 761 demorou mais de 10 segundos
2024-10-16 22:16:16,934 - ERROR - Timeout: A conexão para a linha 788 demorou mais de 10 segundos
2024-10-16 22:16:17,103 - ERROR - Timeout: A conexão para a linha 822 demorou mais de 10 segundos
2024-10-16 22:16:17,306 - ERROR - Timeout: A conexão para a linha 901 demorou mais de 10 segundos
2024-10-16 22:16:17,389 - ERROR - Timeout: A conexão para a linha 870 demorou mais de 10 segundos
2024-10-16 22:16:17,563 - ERROR - Timeout: A conexão para a linha 864 demorou mais de 10 segundos
2024-10-16 22:16:17,651 - ERROR - Timeout: A conexão para a linha 860 demorou mais de 10 segundos
2024-10-16 22:16:17,659 - ERROR - Timeout: A conexão para a linha 721 demorou mais de 10 segundos
2024-10-16 22:16:17,664 - ERROR - Timeout: A conexão para a linha 801 demorou mais de 10 segundos
2024-10-16 22:16:17,

In [38]:
df_concatenado

,COD,REFRESH,LAT_IN_TIME,LON_IN_TIME,CODIGOLINHA,ADAPT,TIPO_VEIC,TABELA,SITUACAO,SITUACAO2,SENT,TCOUNT,SENTIDO_IN_TIME,HORA


In [8]:
import geopy
import requests
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
import logging
import pytz
import threading
import time
import sqlite3
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Conectar ao arquivo .db (ele será criado se não existir)
conn = sqlite3.connect('dados_ocorrencia_onibus.db')
cursor = conn.cursor()
# Função para verificar se a tabela foi criada
def verificar_tabela(nome_tabela):
    cursor.execute("""
        SELECT name FROM sqlite_master WHERE type='table' AND name=?;
    """, (nome_tabela,))
    resultado = cursor.fetchone()

    if resultado:
        logging.info("A tabela '%s' foi criada com sucesso.", nome_tabela)
        return True
    else:
        logging.error("A tabela '%s' não foi encontrada.", nome_tabela)
        return False

# Verificando se a tabela 'onibus' foi criada
if verificar_tabela('onibus'):
    print("Tabela 'onibus' está pronta para uso.")
else:
    print("Erro: Tabela 'onibus' não foi encontrada.")
conn.close()

2024-10-16 20:46:23,734 - INFO - A tabela 'onibus' foi criada com sucesso.


Tabela 'onibus' está pronta para uso.


In [9]:
conn = sqlite3.connect('dados_ocorrencia_onibus.db')
cursor = conn.cursor()
def carregar_tabela_para_dataframe(nome_tabela):
    # Verificar se a tabela existe
    cursor.execute("""
        SELECT name FROM sqlite_master WHERE type='table' AND name=?;
    """, (nome_tabela,))
    
    resultado = cursor.fetchone()

    if resultado:
        # Se a tabela existe, carregar seus dados para um DataFrame
        df = pd.read_sql_query(f"SELECT * FROM {nome_tabela}", conn)
        logging.info("Tabela '%s' carregada com sucesso.", nome_tabela)
        return df
    else:
        logging.error("A tabela '%s' não foi encontrada.", nome_tabela)
        return pd.DataFrame()  # Retornar um DataFrame vazio se a tabela não existir

# Exemplo de uso
df_onibus = carregar_tabela_para_dataframe('onibus')

if not df_onibus.empty:
    print("Dados carregados:")
    print(df_onibus.head())
else:
    print("A tabela 'onibus' está vazia ou não foi encontrada.")
conn.close()

2024-10-16 20:46:40,810 - INFO - Tabela 'onibus' carregada com sucesso.


A tabela 'onibus' está vazia ou não foi encontrada.


In [10]:
df_onibus

,ID,TIPO,LINHA,MENSAGEM,HORA,FLAG_PROCES


In [43]:
df_onibus[df_onibus['TIPO_VEIC'] == 10]

,COD,REFRESH,LAT_IN_TIME,LON_IN_TIME,CODIGOLINHA,ADAPT,TIPO_VEIC,TABELA,SITUACAO,SITUACAO2,SENT,TCOUNT,SENTIDO_IN_TIME,HORA,FLAG_PROCES
10,BB304,15:47,-25.423250,-49.268146,11,1,10,,,,,1,sem tabela,2024-10-16 15:50:02,0
33,BB308,15:49,-25.474023,-49.238706,21,1,10,15,NÃO CONFORMIDADE,REALIZANDO ROTA,CIRCULAR,1,832-TERMINAL CAPAO DA IMBUIA (16:09),2024-10-16 15:50:03,0
49,BC312,15:49,-25.423376,-49.274260,207,1,10,6,NO HORÁRIO,TIPO INCOMPATIVEL,IDA,1,671-PRACA GENERAL OSORIO (16:01),2024-10-16 15:50:03,0
153,BB302,15:50,-25.406575,-49.252471,20,1,10,11,NO HORÁRIO,REALIZANDO ROTA,CIRCULAR,1,819-TERMINAL CABRAL (15:53),2024-10-16 15:50:05,0
158,BB303,15:49,-25.439751,-49.233421,30,1,10,17,ATRASADO,COM MENSAGEM NÃO LIDA,VOLTA,1,950-TERMINAL OFICINAS(SENT.B.ALTO) (16:04),2024-10-16 15:50:05,0
167,BB305,15:49,-25.423621,-49.202291,30,1,10,18,NO HORÁRIO,REALIZANDO ROTA,IDA,1,949-TERMINAL OFICINAS (SENT.CARMO) (16:14),2024-10-16 15:50:05,0
273,BC315,15:50,-25.426276,-49.258910,371,1,10,2,ATRASADO,REALIZANDO ROTA,IDA,1,679-PRACA SANTOS ANDRADE (16:00),2024-10-16 15:50:07,0
274,BC318,15:50,-25.418623,-49.219768,371,1,10,3,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,806-TERMINAL BAIRRO ALTO (16:00),2024-10-16 15:50:07,0
295,CC300,15:49,-25.438523,-49.290050,380,1,10,2,ATRASADO,REALIZANDO ROTA,VOLTA,1,680-PRACA TIRADENTES (15:58),2024-10-16 15:50:07,0
296,HC303,15:49,-25.434445,-49.215475,380,1,10,3,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,97-BAIRRO DETRAN (15:59),2024-10-16 15:50:07,0


In [19]:
from geopy.geocoders import Nominatim

def obter_bairro(lat, lon):
    geolocator = Nominatim(user_agent="meu_app")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    
    if location and 'suburb' in location.raw['address']:
        return location.raw['address']['suburb']
    else:
        return "Bairro não encontrado"

# Exemplo de coordenadas
lat =  -25.4297
lon = -49.2719

bairro = obter_bairro(lat, lon)
print(f"O ônibus está no bairro: {bairro}")


O ônibus está no bairro: Centro
